# Network analysis & GIS

We will introduce two modules in Python: networkx and geopandas. Networkx is used for classical network analysis, while GeoPandas is the extension of Pandas into spatial analysis. As you will see GeoPandas could achieve most of the things the ArcGIS could do. The strength of Geopandas is its computational efficiency, while its weakness is its lack of interactive interface as in ArcGIS. However, the interactive interface in ArcGIS consumes a lot of computational resource, which renders GeoPandas an efficient spatial processing tool in Python.

*   Section 0. Reading files
*   Section 1. Visualizing nodes and edges (GIS)




## Section 0. Reading the files

In [ ]:
# Install packages
!pip install geopandas
!pip install pysal

In [ ]:
pip install matplotlib

In [ ]:
# import modules (old)
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
pd.set_option('display.max_columns', None)

In [ ]:
pip install networkx

In [ ]:
pip install pysal

In [ ]:
# import modules (new)
import geopandas as gpd
import networkx as nx
from pysal.lib import weights
from pysal.lib import cg as geometry

In [ ]:
# define the mounting point on Google drive
#from google.colab import drive
#drive.mount('/content/drive/')

In [ ]:
# Switch to Colab Notebooks.
# Mac system
# !cd '/content/drive/My Drive/Colab Notebooks/data/'
# Windows system
%cd /content/drive/My Drive/Colab Notebooks/data/

In [ ]:
# read the data and shapefile
df = pd.read_csv('Florida_ct.csv', index_col = 0)
florida_shapefile = gpd.read_file('tl_2020_12_tract/tl_2020_12_tract.shp') # read the shapefile

In [ ]:
# view the dataframe
df.head()

In [ ]:
# view the shapefile
florida_shapefile.head()

In [ ]:
# shape of the two files.
print(df.shape)
print(florida_shapefile.shape)

In [ ]:
# quick view of the shapefile.
florida_shapefile.plot(figsize = (10,10))
plt.show()

## Preprocessing the data

In [ ]:
# adjust the object types to facilitate the merge
florida_shapefile['GEOID'] = florida_shapefile.GEOID.astype('int64')

In [ ]:
# combine the dataframe with the shapefile.
# Note that it is important to choose how - e.g., inner, right, left, etc. Here I choose 'left' for teaching purposes.
df_shp = florida_shapefile.merge(df,
                                 how = 'left',
                                 left_on = 'GEOID',
                                 right_on = 'full_ct_fips')


In [ ]:
# check the shape of the new file.
# It combines the census data set and the shapefile.
df_shp.shape
# 101 = 88 + 13

In [ ]:
df_shp.head()

In [ ]:
pip install openpyxl


In [ ]:
df_shp['geometry']


In [ ]:
import pandas as pd
import numpy as np
from shapely.geometry import Polygon, Point


# 获取表中第一个多边形
first_polygon = df_shp['geometry'].iloc[0]

# 定义在多边形内生成的点的数量
num_points = 10

# 生成一系列分散的经度和纬度坐标，并将它们添加到列表中
points = []
while len(points) < num_points:
    # 生成随机经度和纬度坐标
    longitude = np.random.uniform(first_polygon.bounds[0], first_polygon.bounds[2])
    latitude = np.random.uniform(first_polygon.bounds[1], first_polygon.bounds[3])
    point = Point(longitude, latitude)
    # 检查点是否位于第一个多边形内部
    if point.within(first_polygon):
        points.append((longitude, latitude))

# 将坐标数据保存到DataFrame
df = pd.DataFrame(points, columns=['longitude', 'latitude'])

# 将DataFrame保存到Excel文件
file_path = 'coordinate.xlsx'
df.to_excel(file_path, index=False)

print("数据已保存到文件:", file_path)


In [ ]:
# With the current approach, I will fill in ZEROS into the NaN values.
# However, it is NOT necessarily the best approach.
df_shp = df_shp.fillna(0.0)

# Section 1. Visualizing nodes and edges in a spatial network.

GIS practice is a specific case of the general network analysis, certainly with its spcialization. In most of the cases, people use some spatial units as the unit of analysis, and census tract is one of the most common spatial units. Therefore, the GIS visulization is the same as visualizing the node features of a spatial graph.

In [ ]:
# example 1.0
# visualizing property values of the whole Florida
fig, ax = plt.subplots(figsize=(8, 8))

ax.axis('off') # remove the axis
df_shp.plot(column = 'property_value_median', cmap = 'plasma', legend=True,
            legend_kwds={'label': "Property Values", 'orientation': "vertical", 'shrink': 0.3},
            ax = ax)
ax.set_title('Median Property Values in Florida')

plt.tight_layout()
plt.show()

In [ ]:
# example 1.1. adjusting the legend vmin and vmax to highlight the areas with high property values.
# visualizing property values of the whole Florida.

fig, ax = plt.subplots(figsize=(8, 8))

ax.axis('off') # remove the axies
df_shp.plot(column = 'property_value_median', cmap = 'magma', legend=True, alpha = 1.0,
            vmin = 50000, vmax = 500000,
            legend_kwds={'label': "Property Values", 'orientation': "vertical", 'shrink': 0.3},
            ax = ax)
ax.set_title('Median Property Values in Florida')

plt.tight_layout()
plt.show()


## **Exercise** Visualizing household income in Florida

**How to zoom into a certain region?** Two approaches

In [ ]:
# example 2.1. use the Florida shapefile but adjust the longitude and latitude to show a small area.
# x is the longitude.
# y is the latitude.

x_min = -82.649702
y_max = 29.827481
x_max = -82.025303
y_min = 29.428041

fig, ax = plt.subplots(figsize=(8, 8))

ax.axis('off') # remove the axies
# df_shp.plot(facecolor="None", edgecolor='black', linewidth=0.1, ax = ax)
df_shp.plot(column = 'property_value_median', cmap = 'magma', legend=True, alpha = 1.0,
            vmin = 50000, vmax = 300000,
            legend_kwds={'label': "Property Values", 'orientation': "vertical", 'shrink': 0.3},
            ax = ax)
ax.set_title('Median Property Values around Gainesville')

ax.set_xlim(x_min, x_max)
ax.set_ylim(y_min, y_max)

plt.tight_layout()
plt.show()


In [ ]:
# example 2.2. Visualize only a subset.
# create a subset of the full data set.
Alachua_shp = df_shp.loc[df_shp.COUNTYFP == '001', :].reset_index()
print("The size of the Alachua shapefile is: ", Alachua_shp.shape)

# create a column for the coordinates
Alachua_shp['coords'] = Alachua_shp['geometry'].apply(lambda x: x.representative_point().coords[:])
Alachua_shp['coords'] = [coords[0] for coords in Alachua_shp['coords']]

In [ ]:
# example 2.2. visualizing only the Alachua county.
fig, ax = plt.subplots(figsize=(8, 8))

ax.axis('off') # remove the axies
# df_shp.plot(facecolor="None", edgecolor='black', linewidth=0.1, ax = ax)
Alachua_shp.plot(column = 'property_value_median', cmap = 'magma', legend=True, alpha = 1.0,
            vmin = 50000, vmax = 300000,
            legend_kwds={'label': "Property Values", 'orientation': "vertical", 'shrink': 0.3},
            ax = ax)
ax.set_title('Median Property Values in Alachua County')

plt.tight_layout()
plt.show()

## Creating edges, check the adjacency matrix, and visualizing edges.

In [ ]:
# example 3.1. visualizing only the Alachua county.
fig, ax = plt.subplots(figsize=(10, 10))

ax.axis('off') # remove the axies
# df_shp.plot(facecolor="None", edgecolor='black', linewidth=0.1, ax = ax)
Alachua_shp.plot(color = 'white', edgecolor = 'black', ax = ax)

for idx, row in Alachua_shp.iterrows():
  plt.annotate(s=idx, horizontalalignment='center', color='blue', xy=row['coords'])

ax.set_title('Alachua County')

plt.tight_layout()
plt.show()

In [ ]:
# Creating the adjacency matrix.
w_queen = weights.contiguity.Queen.from_dataframe(Alachua_shp)

In [ ]:
# Visualizing the adjacency matrix (20*20)
# the matrix view of the adjacency matrix.

w_queen_df = pd.DataFrame(w_queen.full()[0],
                          columns = w_queen.full()[1],
                          index = w_queen.full()[1])

# set up the printing option
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

# printing only the first 20 columns & rows
w_queen_df.iloc[:20, :20]

In [ ]:
# Preparing for visualizing the edges by reprojecting Alachua shapefile
Alachua_shp = Alachua_shp.to_crs('3514')

# edit the coordinate column
Alachua_shp['coords'] = Alachua_shp['geometry'].apply(lambda x: x.representative_point().coords[:])
Alachua_shp['coords'] = [coords[0] for coords in Alachua_shp['coords']]

In [ ]:
# visualizing the edges on a map
fig, ax = plt.subplots(figsize=(10, 10))

ax.axis('off') # remove the axies

# 3514 is the projection index for the Alachua County
Alachua_shp.plot(color = 'white', edgecolor = 'black', ax = ax)
w_queen.plot(Alachua_shp, ax = ax,
             edge_kws=dict(color='r', linestyle=':', linewidth=1),
             node_kws=dict(marker='o', color='r'))

# edit the coordinates
# create a column for the coordinates

for idx, row in Alachua_shp.iterrows():
  plt.annotate(s=idx, horizontalalignment='center', color='blue', xy=row['coords'])

ax.set_title('Median Property Values in Alachua County')

plt.tight_layout()
plt.show()

## **Exercise.** Create a subset for Miami-Dade County. Visualize the property values, create the adjacency matrix, and visualize the edges.